In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
import numpy

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
df = pd.read_csv('data/ner_dataset_dota2_chat.csv', encoding = "utf_8", sep=';')

In [3]:
df.head(10)

,Sentence #,Word,Tag
0,Sentence: 1,yes,O
1,NaN,dog,B-bad
2,Sentence: 2,yeah,O
3,Sentence: 3,fast,O
4,NaN,and,O
5,NaN,furious,O
6,Sentence: 4,too,O
7,NaN,fas,O
8,Sentence: 5,haha,O
9,Sentence: 6,sad,O


In [4]:
df.describe()

,Sentence #,Word,Tag
count,241,586,587
unique,241,341,7
top,Sentence: 201,game,O
freq,1,15,456


In [5]:
df['Tag'].unique()

array(['O', 'B-bad', 'I-pra', 'B-pra', 'I-bad', 'B-her', 'I-her'],
      dtype=object)

In [6]:
df.isnull().sum()

Sentence #    346
Word            1
Tag             0
dtype: int64

In [7]:
df = df.fillna(method = 'ffill')

In [11]:
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p) for w, p in zip(s['Word'].values.tolist(),
                                                 s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [12]:
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'yes dog'

In [13]:
sent = getter.get_text()
print(sent)

[('yes', 'O'), ('dog', 'B-bad')]


In [14]:
sentences = getter.sentences

In [18]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [19]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [21]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

/Users/arrivaldwis/Library/Python/3.7/lib/python/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [22]:
import pickle

In [23]:
saved_model = pickle.dumps(crf)
crf_from_pickle = pickle.loads(saved_model)
crf_from_pickle.predict(X_test)

[['B-pra', 'B-pra'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['B-pra', 'I-pra'],
 ['O'],
 ['O', 'O', 'O'],
 ['O'],
 ['B-pra', 'I-pra'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'B-bad'],
 ['O', 'O', 'O', 'O'],
 ['O', 'O'],
 ['O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['B-pra', 'I-pra'],
 ['O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O'],
 ['O'],
 ['O', 'O', 'O', 'O'],
 ['O', 'O'],
 ['O'],
 ['O', 'O'],
 ['B-pra', 'I-pra'],
 ['O'],
 ['O'],
 ['O

In [24]:
from sklearn.externals import joblib

/Users/arrivaldwis/Library/Python/3.7/lib/python/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [25]:
joblib.dump(crf, 'toxicner.pkl')

['toxicner.pkl']

In [26]:
# Load model ner dari pkl file
ner_from_joblib = joblib.load('toxicner.pkl')  
  
# Menggunakan model untuk melakukan prediksi
ner_from_joblib.predict(X_test)

[['B-pra', 'B-pra'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['B-pra', 'I-pra'],
 ['O'],
 ['O', 'O', 'O'],
 ['O'],
 ['B-pra', 'I-pra'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'B-bad'],
 ['O', 'O', 'O', 'O'],
 ['O', 'O'],
 ['O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['B-pra', 'I-pra'],
 ['O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O'],
 ['O', 'O', 'O'],
 ['O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O', 'O'],
 ['O'],
 ['O', 'O', 'O', 'O'],
 ['O', 'O'],
 ['O'],
 ['O', 'O'],
 ['B-pra', 'I-pra'],
 ['O'],
 ['O'],
 ['O

In [27]:
y_pred = crf.predict(X_test)

In [28]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.8115688530878403


In [29]:
report = flat_classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

       B-bad       1.00      0.25      0.40         4
       B-her       0.00      0.00      0.00         5
       B-pra       0.67      0.33      0.44        12
       I-her       0.00      0.00      0.00         1
       I-pra       0.75      0.38      0.50         8
           O       0.86      0.98      0.92       128

    accuracy                           0.85       158
   macro avg       0.55      0.32      0.38       158
weighted avg       0.81      0.85      0.81       158



/Users/arrivaldwis/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
